## TODO 
- [ ] check the galaxy counts expected for LSST 
- [x] find the equations for the effect of magnification on the redshift distriution
- [ ] include the redshift information in the model
- [ ] check the galaxy counts expected for DESI
- [ ] check the problem of radius limit in modelling

# Bibio 
- https://arxiv.org/pdf/astro-ph/9912508.pdf
- https://iopscience.iop.org/article/10.1088/0004-637X/772/1/65/pdf
- https://arxiv.org/pdf/astro-ph/9511150.pdf
- https://arxiv.org/pdf/astro-ph/9406052.pdf
- faint end slope https://iopscience.iop.org/article/10.1086/505455/pdf, https://www.aanda.org/articles/aa/pdf/2003/13/aa3018.pdf

In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
import sys
sys.path.append('./support')
import clmm
import clmm.modeling as mod
import scipy.integrate as integrate
import mock_data as mock

In [13]:
import GCRCatalogs
from GCR import GCRQuery

In [14]:
from astropy.cosmology import FlatLambdaCDM
astropy_cosmology_object = FlatLambdaCDM(H0=70, Om0=0.27, Ob0=0.045)
cosmo_ccl = mod.cclify_astropy_cosmo(astropy_cosmology_object)

In [15]:
#catalog = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image_photoz_calib')

In [16]:
density_profile_parametrization = 'nfw'
mass_Delta = 200
cluster_mass = 1.5e14
cluster_concentration = 4
z_cl = 0.2
z_source = 2
ngals = 1000

In [17]:
ideal_with_src_dist = mock.generate_galaxy_catalog(
    cluster_mass, z_cl, cluster_concentration, cosmo_ccl, ngals, mass_Delta, 'chang13', zsrc_min=z_cl + 0.2,
    zsrc_max=7.0)

In [45]:
r3d = np.loadtxt("../tests/data/numcosmo/radial_profiles.txt", dtype = float, usecols=0)

In [51]:
r3dA = np.loadtxt("../tests/data/numcosmo/radial_profiles.txt", dtype = float)[:,0]

In [61]:
r3dA[1].dtype

dtype('float64')

In [62]:
r3d[1].dtype

dtype('float64')

In [ ]:
from astropy.io import ascii
dat = ascii.read("profiles_numcosmo.txt")

r3d = dat['r3d']

In [20]:
def plot_profile(r, profile_vals, profile_label='rho', label=None):
    plt.loglog(r, profile_vals, label=label)
    plt.xlabel('r [Mpc]', fontsize='xx-large')
    plt.ylabel(profile_label, fontsize='xx-large')

In [42]:
rho = mod.get_3d_density(r3d, mdelta=cluster_mass, cdelta=cluster_concentration, 
                       z_cl=z_cl, cosmo=cosmo_ccl)

In [43]:
rho

array([ 1.77127618e+16,  0.00000000e+00,  6.02658808e+01,  1.52249322e+04,
        1.99324104e+14,  2.49198351e+12, -2.08089304e+15,  1.60306892e+16,
        0.00000000e+00,  6.58879972e+01,  1.52608613e+04,  1.99631435e+14,
        2.70059939e+12, -1.22447280e+15,  1.44991907e+16,  0.00000000e+00,
        7.22214083e+01,  1.53027149e+04,  1.99989052e+14,  2.93283032e+12,
       -7.34307967e+14,  1.31049830e+16,  0.00000000e+00,  7.93797167e+01,
        1.53514343e+04,  2.00404802e+14,  3.19203208e+12, -4.43257167e+14,
        1.18359683e+16,  0.00000000e+00,  8.74983284e+01,  1.54081043e+04,
        2.00887683e+14,  3.48212919e+12, -2.66447395e+14,  1.06811292e+16,
        0.00000000e+00,  9.67394403e+01,  1.54739754e+04,  2.01448002e+14,
        3.80772564e+12, -1.57808404e+14,  9.63043296e+15,  0.00000000e+00,
        1.07298091e+02,  1.55504878e+04,  2.02097548e+14,  4.17423959e+12,
       -9.10015071e+13,  8.67474341e+15,  0.00000000e+00,  1.19409937e+02,
        1.56393009e+04,  

In [ ]:
plot_profile(r3d, rho, "$\\rho$", 'CLMM')
plot_profile(r3d, dat['rho'], "$\\rho$", 'numcosmo')

In [ ]:
plt.plot(r3d, (rho-dat['rho'])/rho,'.')
plt.xscale('log')

In [ ]:
Sigma = mod.predict_surface_density(r3d, cluster_mass, cluster_concentration, z_cl, cosmo=cosmo_ccl, 
                                  delta_mdef=mass_Delta, 
                                  halo_profile_model=density_profile_parametrization)

In [ ]:
plot_profile(r3d, Sigma, "$\\rho$", 'CLMM')
plot_profile(r3d, dat['Sigma'], "$\\rho$", 'numcosmo')

In [ ]:
plt.plot(r3d, (Sigma-dat['Sigma'])/Sigma,'.')
plt.xscale('log')

In [ ]:
DeltaSigma = mod.predict_excess_surface_density(r3d, cluster_mass, cluster_concentration, z_cl, cosmo=cosmo_ccl, 
                                              delta_mdef=mass_Delta, 
                                              halo_profile_model=density_profile_parametrization)

In [ ]:
plot_profile(r3d, DeltaSigma, "$\\rho$", 'CLMM')
plot_profile(r3d, dat['DeltaSigma'], "$\\rho$", 'numcosmo')

In [ ]:
plt.plot(r3d, (DeltaSigma-dat['DeltaSigma'])/DeltaSigma,'.')
plt.xscale('log')

In [ ]:
gammat = mod.predict_tangential_shear(r3d, mdelta=cluster_mass, cdelta=cluster_concentration, z_cluster=z_cl, 
                                    z_source=z_source, cosmo=cosmo_ccl, delta_mdef=mass_Delta, 
                                    halo_profile_model=density_profile_parametrization, 
                                    z_src_model='single_plane')

In [ ]:
gammat = mod.predict_tangential_shear(r3d, mdelta=cluster_mass, cdelta=cluster_concentration, z_cluster=z_cl, 
                                    z_source=0.01, cosmo=cosmo_ccl, delta_mdef=mass_Delta, 
                                    halo_profile_model=density_profile_parametrization, 
                                    z_src_model='single_plane')

In [ ]:
import warnings

In [ ]:
warnings.warn

In [ ]:
z = np.linspace(0,5)

In [ ]:
[z]

In [ ]:
np.sum(z<z_cl)!=0

In [ ]:
plot_profile(r3d, gammat, "$\\rho$", 'CLMM')
plot_profile(r3d, dat['gammat'], "$\\rho$", 'numcosmo')

In [ ]:
plt.plot(r3d, (gammat-dat['gammat'])/gammat,'.')
plt.xscale('log')

In [ ]:
kappa = mod.predict_convergence(r3d, mdelta=cluster_mass, cdelta=cluster_concentration, 
                               z_cluster=z_cl, z_source=z_source,
                               cosmo=cosmo_ccl, delta_mdef=mass_Delta, 
                               halo_profile_model=density_profile_parametrization, 
                               z_src_model='single_plane')

In [ ]:
plot_profile(r3d, kappa, "$\\rho$", 'CLMM')
plot_profile(r3d, dat['kappa'], "$\\rho$", 'numcosmo')

In [ ]:
plt.plot(r3d, (kappa-dat['kappa'])/kappa,'.')
plt.xscale('log')

In [ ]:
gt = mod.predict_reduced_tangential_shear(r3d, mdelta=cluster_mass, cdelta=cluster_concentration, 
                                        z_cluster=z_cl, z_source=z_source, cosmo=cosmo_ccl, 
                                        delta_mdef=mass_Delta, 
                                        halo_profile_model=density_profile_parametrization, 
                                        z_src_model='single_plane')

In [ ]:
plot_profile(r3d, gt, "$\\rho$", 'CLMM')
plot_profile(r3d, dat['gt'], "$\\rho$", 'numcosmo')

In [ ]:
plt.plot(r3d, (gt-dat['gt'])/kappa,'.')
plt.xscale('log')

In [ ]:
mu = mod.predict_magnification(r3d, mdelta=cluster_mass, cdelta=cluster_concentration, 
                                        z_cluster=z_cl, z_source=z_source, cosmo=cosmo_ccl, 
                                        delta_mdef=mass_Delta, 
                                        halo_profile_model=density_profile_parametrization, 
                                        z_src_model='single_plane')

In [ ]:
plot_profile(r3d, mu, "$\\rho$", 'CLMM')
plot_profile(r3d, dat['mag'], "$\\rho$", 'numcosmo')

In [ ]:
plt.plot(r3d, (mu-dat['mag'])/mu,'.')
plt.ylabel('1 - $\mu_{Nc} / \mu_{CLMM} $', fontsize='xx-large')
plt.xlabel('$r[Mpc]$', fontsize='xx-large')
plt.xscale('log')
plt.grid()

## Magnification definition

#in the WL limit

def mu_wl(kappa): 
    "magnification with WL approximation"
    mu_wl = 1 + 2*kappa
    return mu_wl

In [ ]:
#arbitrary Wl/SL limit : when the convergence is k>0.1
WL_limit_radius= r3d[kappa<0.1][0]

magnification = mu(kappa,gammat)
magnification_wl = mu_wl(kappa)

In [ ]:
plot_profile(r3d,mu, '$\mu$', "no approx.")
#plot_profile(r3d,magnification_wl, '$\mu$', "WL approx.")
plt.axvline(WL_limit_radius,color='black', linestyle='dashed', label="r inside which $\\kappa>0.1$")
plt.legend()
plt.yscale('linear')

## Magnification bias definition

In [ ]:
#magnification bias : number of lensed source over the number of unlensed source
#beta = slope of the power law luminosity function around the limiting flux of the survey, with N ~ AS^(-beta)

def mu_bias(mu,beta):
    mu_bias = mu**(beta-1)
    return mu_bias

In [ ]:
#For early type galaxies

alpha_ET = 0.6
beta_ET = alpha_ET * 2.5

plt.plot(r3d, mu_bias(mu,beta_ET))
#plt.plot(r3d, mu_bias(magnification_wl,beta_ET))
plt.xscale('log')
plt.axhline(1,color='black',alpha=0.2)
plt.axvline(WL_limit_radius,color='black', linestyle='dashed', label="r inside which $\\kappa>0.1$")

In [ ]:
#For late type galaxies

alpha_LT = -1.30
beta_LT = alpha_LT * 2.5

plt.plot(r3d, mu_bias(mu,beta_LT))
#plt.plot(r3d, mu_bias(magnification_wl,beta_LT))
plt.xscale('log')
plt.axhline(1,color='black',alpha=0.2)
plt.axvline(WL_limit_radius,color='black', linestyle='dashed', label="r inside which $\\kappa>0.1$")

## Magnification bias in redshift distribution

### Redshift ditribution

In [ ]:
#definition of a redshift distribution following Chang et al. 2013 arXiv:1305.0793

def neff(z):
    """Redshift distribution function"""
    neff_0, alpha, beta, z0 = 30, 1.24, 1.01, 0.51
    return neff_0 * (z**alpha)*np.exp(-(z/z0)**beta)

In [ ]:
#definition of a redshift distribution following Chang et al. 2013 arXiv:1305.0793

def pdf_z(z):
    """Redshift distribution function"""
    alpha, beta, z0 = 1.24, 1.01, 0.51
    return (z**alpha)*np.exp(-(z/z0)**beta)

In [ ]:
z = np.linspace(0.2,5,10000)[1:]

In [ ]:
plt.plot(z, neff(z),'k', label ='unlensed')
plt.plot(z, (neff(z) * mu_bias(mu_2d, beta_ET))[25,:],'r', label ='unlensed early type galaxies')
plt.plot(z, (neff(z) * mu_bias(mu_2d, beta_ET))[50,:],'r--', label ='unlensed early type galaxies')
plt.plot(z, neff(z) * mu_bias(mu_2d, beta_LT)[50,:],'b', label ='lensed late type galaxies')
plt.legend()
plt.xlabel('z');
plt.ylabel('neff [arcmin$^{-2}$]')

In [ ]:
plt.plot(z, (neff(z) * mu_bias(mu_2d, beta_ET))[25,:] - neff(z))

In [ ]:
#cheap way of doing mean
zmean = np.sum(neff(z)*z)/np.sum(neff(z))

In [ ]:
#cheap way of doing mean
zmean_lensed_ET = np.zeros(r3d.size)
for i in range(r3d.size):
    zmean_lensed_ET[i] = np.sum((neff(z) * mu_bias(mu_2d, beta_ET))[i,:]*z)/np.sum((neff(z) * mu_bias(mu_2d, beta_ET))[25,:])

In [ ]:
zmean_lensed_ET/zmean

In [ ]:
plt.plot(r3d, zmean_lensed_ET/zmean - 1, 'k.')
plt.xscale('log')
plt.grid()
plt.axhline(0)

In [ ]:
#mean redshift of the distribution

def z_mean(func, a, b):
    num = integrate.quad(lambda x: func(x)*x, a, b)
    den = integrate.quad(lambda x: func(x), a, b)
    return num[0]/den[0]

In [ ]:
z_mean(neff,0,5)

In [ ]:
def test(z):
    test = (neff(z) * mu_bias(mu_2d, beta_ET))[25,:]
    return test

In [ ]:
mu_2d.shape

In [ ]:
mu_bias(mu_2d,beta_ET)

In [ ]:
(z * neff(z) * mu_bias(mu_2d, beta_ET))

In [ ]:
#here we make the simplification that their is the same number if sources galaxies in each bin and \
#that they have the same redshift distribution
mu_2d = np.zeros((r3d.size, z.size))

for i in range(r3d.size):
    mu_2d[i,:] = mod.predict_magnification(r3d[i], mdelta=cluster_mass, cdelta=cluster_concentration, 
                                        z_cluster=z_cl, z_source=z, cosmo=cosmo_ccl, 
                                        delta_mdef=mass_Delta, 
                                        halo_profile_model=density_profile_parametrization, 
                                        z_src_model='single_plane')

In [ ]:
plt.loglog(r3d, np.nanmean(mu_2d,axis=1), 'r')
plot_profile(r3d,mu, '$\mu$', "no approx.")

In [ ]:
plt.plot(z, np.nanmean(mu_2d,axis=0), 'r')
plt.grid()
plt.axvline(0.2)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(mu_2d,origin="lower",vmin=-1, vmax=10, extent=[z.min(),z.max(),r3d.min(), r3d.max()],cmap='seismic')
plt.yscale('log')
plt.colorbar()

In [ ]:
z_cl

In [ ]:
test = np.random.choice(z, size = 20, p = pdf_z(z)/np.sum(pdf_z(z)))

In [ ]:
test = np.random.choice(z, size = 1000, p = pdf_z(z)/np.sum(pdf_z(z)))

In [ ]:
ideal_with_src_dist = mock.generate_galaxy_catalog(
    cluster_mass, z_cl, cluster_concentration, cosmo_ccl, ngals, mass_Delta, 'chang13', zsrc_min=z_cl + 0.2,
    zsrc_max=7.0)

In [ ]:
def trunc_pzfxn(z, zmin):
    """Truncated redshift distribution function"""
    return np.heaviside((z>zmin),0)*pzfxn(z)

In [ ]:
z = np.linspace(0,5)

plt.hist(ideal_with_src_dist['z'],bins=+20, density=True, stacked=True);
plt.plot(z, trunc_pzfxn(z, z_cl+0.2),'k', label ='unlensed')

In [ ]:
plt.plot(z, pzfxn(z),'k', label ='unlensed')
plt.plot(z, pzfxn(z) * mu_bias(mu[np.argwhere(r3d>0.1)][0],beta_ET),'r', label ='unlensed early type galaxies')
plt.plot(z, pzfxn(z) * mu_bias(mu[np.argwhere(r3d>0.1)][0],beta_LT),'b', label ='lensed late type galaxies')
plt.legend()

### Compute mean of the distribution

In [ ]:
#mean redshift of the distribution

def z_mean(func, a, b):
    num = integrate.quad(lambda x: func(x)*x, a, b)
    den = integrate.quad(lambda x: func(x), a, b)
    return num[0]/den[0]

In [ ]:
def my_pzfxn(z):
    return trunc_pzfxn(z,z_cl+0.2)

In [ ]:
z_mean(my_pzfxn, 0, np.inf)

In [ ]:
z_mean(pzfxn, 0.1, 3)

In [ ]:
np.mean(ideal_with_src_dist['z'])

### Compute the magnification of sources that follow a redshift distribution

In [ ]:
#here we make the simplification that their is the same number if sources galaxies in each bin and \
#that they have the same redshift distribution
mu_2d = np.zeros((r3d.size, ngals))

for i in range(r3d.size):
    mu_2d[i,:] = mod.predict_magnification(r3d[i], mdelta=cluster_mass, cdelta=cluster_concentration, 
                                        z_cluster=z_cl, z_source=ideal_with_src_dist['z'], cosmo=cosmo_ccl, 
                                        delta_mdef=mass_Delta, 
                                        halo_profile_model=density_profile_parametrization, 
                                        z_src_model='single_plane')

In [ ]:
plt.loglog(r3d, np.nanmean(mu_2d,axis=1), 'r')
plot_profile(r3d,mu, '$\mu$', "no approx.")

In [ ]:
pzfxn(z) * mu_bias(mu[np.argwhere(r3d>0.1)][0],beta_LT)

In [ ]:
# THIS IS INCORRECT

plt.hist(ideal_with_src_dist['z'],bins=+20, density=True, stacked=True,label='unlensed',color='yellow');
plt.hist(ideal_with_src_dist['z']* mu_bias(np.nanmean(mu_2d,axis=0),beta_ET),bins=+20, density=True, stacked=True,alpha=0.2, label='lensed ET');
plt.hist(ideal_with_src_dist['z']* mu_bias(np.nanmean(mu_2d,axis=0),beta_LT),bins=+20, density=True, stacked=True,alpha=0.2, label='lensed LT');
plt.legend()

In [ ]:
plt.plot(z, pzfxn(z),'k', label ='unlensed')
plt.plot(z, pzfxn(z) * mu_bias(np.nanmean(mu_2d,axis=0),beta_ET),'r', label ='unlensed early type galaxies')
plt.plot(z, pzfxn(z) * mu_bias(np.nanmean(mu_2d,axis=0),beta_LT),'b', label ='lensed late type galaxies')
plt.legend()

In [ ]:
z_mean(pzfxn, 0.4, np.inf)

In [ ]:
np.mean(ideal_with_src_dist['z'])

In [ ]:
np.mean(ideal_with_src_dist['z']* mu_bias(np.nanmean(mu_2d,axis=0),beta_ET))

In [ ]:
np.mean(ideal_with_src_dist['z']* mu_bias(np.nanmean(mu_2d,axis=0),beta_LT))

### TESTS

In [ ]:
def beta_z(z, beta_0,beta_1):
    beta = beta_0 + beta_1 * (1+z)
    return beta


In [ ]:
def mu_bias(z, mu, beta_0, beta_1):
    mu_bias = mu**(beta_z(z,beta_0, beta_1) -1)
    return mu_bias

In [ ]:
beta_0, beta_1 = 0.9, 0.1
mu_bias(z, magnification[np.argwhere(r3d>0.1)][0], beta_0, beta_1)

In [ ]:
def fct(z):
    return pzfxn(z)*mu_bias(z, magnification[np.argwhere(r3d>0.1)][0], beta_0, beta_1)

In [ ]:
z_mean(fct, 0, np.inf)